In [ ]:
import pypylon.pylon as py
import os
import cv2
import time

# Initialize Pylon Camera
tlf = py.TlFactory.GetInstance()
devices = tlf.EnumerateDevices()
cam = py.InstantCamera(tlf.CreateDevice(devices[0]))

cam.Open()

# Configure Camera
cam.Width.Value = 1920
cam.Height.Value = 1080
cam.AcquisitionFrameRateEnable.Value = True
cam.AcquisitionFrameRate.Value = 200

cam.TriggerMode.Value = "On"  
cam.TriggerSource.Value = "Line1"  
cam.TriggerActivation.Value = "RisingEdge"
cam.TriggerSelector.Value = "FrameStart"

# cam.ExposureMode.Value = "TriggerControlled"  

cam.StartGrabbing(py.GrabStrategy_OneByOne)

# Setup video writer
op_name = 'output.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(op_name, fourcc, fps=100, frameSize=(1920, 1080))

fcount = 0

while cam.IsGrabbing():
    res = cam.RetrieveResult(5000, py.TimeoutHandling_Return)

    # Check if the frame was grabbed successfully
    if not res.GrabSucceeded():
        print("Waiting for hardware trigger...")
        time.sleep(0.5)  # Small delay before retrying
        continue  # Retry without breaking the loop
    
    # If trigger is received, process the frame
    fcount += 1
    print(f"Frame num: {fcount}")
    video_writer.write(res.Array)

    res.Release()

# Stop and release resources
cam.StopGrabbing()
cam.Close()
video_writer.release()
